In [ ]:
from sentence_transformers import InputExample, losses, util, SentenceTransformer, CrossEncoder
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm

<h3>Only run the following 2 cells, the first time the program is executed.</h3>
We are creating upfront the auxiliar databases to increase predicting speed. After doing this, we can skip this step and just load them for inference.

In [ ]:
data_folder = '/home/datasets/'
ALL_DATA= pd.read_csv(data_folder + "database.csv", encoding="utf-8")


indexes_CP_1 = ALL_DATA['First Digit CP4'].loc[lambda x: x==1].index
indexes_CP_2 = ALL_DATA['First Digit CP4'].loc[lambda x: x==2].index
indexes_CP_3 = ALL_DATA['First Digit CP4'].loc[lambda x: x==3].index
indexes_CP_4 = ALL_DATA['First Digit CP4'].loc[lambda x: x==4].index
indexes_CP_5 = ALL_DATA['First Digit CP4'].loc[lambda x: x==5].index
indexes_CP_6 = ALL_DATA['First Digit CP4'].loc[lambda x: x==6].index
indexes_CP_7 = ALL_DATA['First Digit CP4'].loc[lambda x: x==7].index
indexes_CP_8 = ALL_DATA['First Digit CP4'].loc[lambda x: x==8].index
indexes_CP_9 = ALL_DATA['First Digit CP4'].loc[lambda x: x==9].index

In [ ]:
model = SentenceTransformer('/home/bi_encoder tuned')
DATABASE_Embeddings = ALL_DATA['Morada_Full_Normalizada'].to_list()
DATABASE_Embeddings = model.encode(DATABASE_Embeddings) # Isto retorna ndArray

#Write CSV's of DB and all the Filtered by CP4
DATABASE_Embeddings_pd = pd.DataFrame(DATABASE_Embeddings); DATABASE_Embeddings_pd.to_csv(data_folder + 'DB_Embeddings_Full.csv', index=False)
DB_CP1_ = DATABASE_Embeddings_pd.iloc[indexes_CP_1,:]; DB_CP1_.to_csv(data_folder + 'DB_Embeddings_1.csv', index=False)
DB_CP2_ = DATABASE_Embeddings_pd.iloc[indexes_CP_2,:]; DB_CP2_.to_csv(data_folder + 'DB_Embeddings_2.csv', index=False)
DB_CP3_ = DATABASE_Embeddings_pd.iloc[indexes_CP_3,:]; DB_CP3_.to_csv(data_folder + 'DB_Embeddings_3.csv', index=False)
DB_CP4_ = DATABASE_Embeddings_pd.iloc[indexes_CP_4,:]; DB_CP4_.to_csv(data_folder + 'DB_Embeddings_4.csv', index=False)
DB_CP5_ = DATABASE_Embeddings_pd.iloc[indexes_CP_5,:]; DB_CP5_.to_csv(data_folder + 'DB_Embeddings_5.csv', index=False)
DB_CP6_ = DATABASE_Embeddings_pd.iloc[indexes_CP_6,:]; DB_CP6_.to_csv(data_folder + 'DB_Embeddings_6.csv', index=False)
DB_CP7_ = DATABASE_Embeddings_pd.iloc[indexes_CP_7,:]; DB_CP7_.to_csv(data_folder + 'DB_Embeddings_7.csv', index=False)
DB_CP8_ = DATABASE_Embeddings_pd.iloc[indexes_CP_8,:]; DB_CP8_.to_csv(data_folder + 'DB_Embeddings_8.csv', index=False)
DB_CP9_ = DATABASE_Embeddings_pd.iloc[indexes_CP_9,:]; DB_CP9_.to_csv(data_folder + 'DB_Embeddings_9.csv', index=False)

<h3>After Pre-Embedding Step</h3>
Assuming auxiliar databases are created, let's load them, load the test dataset and start inference

In [ ]:
data_folder = '/home/datasets/'

test_data = pd.read_csv(data_folder + "test_data.csv")
ALL_DATA= pd.read_csv(data_folder + "database.csv", encoding="utf-8")
DATABASE_Embeddings= pd.read_csv(data_folder + "DB_embeddings_Full.csv")

DB_CP1_ = pd.read_csv(data_folder + "DB_Embeddings_1.csv")
DB_CP2_ = pd.read_csv(data_folder + "DB_Embeddings_2.csv")
DB_CP3_ = pd.read_csv(data_folder + "DB_Embeddings_3.csv")
DB_CP4_ = pd.read_csv(data_folder + "DB_Embeddings_4.csv")
DB_CP5_ = pd.read_csv(data_folder + "DB_Embeddings_5.csv")
DB_CP6_ = pd.read_csv(data_folder + "DB_Embeddings_6.csv")
DB_CP7_ = pd.read_csv(data_folder + "DB_Embeddings_7.csv")
DB_CP8_ = pd.read_csv(data_folder + "DB_Embeddings_8.csv")
DB_CP9_ = pd.read_csv(data_folder + "DB_Embeddings_9.csv")


#Convert Embeddings from Pandas DF to numpy and float
DATABASE_Embeddings = DATABASE_Embeddings.to_numpy().astype(float)
DB_CP1_ = DB_CP1_.to_numpy().astype(float)
DB_CP2_ = DB_CP2_.to_numpy().astype(float)
DB_CP3_ = DB_CP3_.to_numpy().astype(float)
DB_CP4_ = DB_CP4_.to_numpy().astype(float)
DB_CP5_ = DB_CP5_.to_numpy().astype(float)
DB_CP6_ = DB_CP6_.to_numpy().astype(float)
DB_CP7_ = DB_CP7_.to_numpy().astype(float)
DB_CP8_ = DB_CP8_.to_numpy().astype(float)
DB_CP9_ = DB_CP9_.to_numpy().astype(float)



#Load Indexes
indexes_CP_1 = ALL_DATA['First Digit CP4'].loc[lambda x: x==1].index
indexes_CP_2 = ALL_DATA['First Digit CP4'].loc[lambda x: x==2].index
indexes_CP_3 = ALL_DATA['First Digit CP4'].loc[lambda x: x==3].index
indexes_CP_4 = ALL_DATA['First Digit CP4'].loc[lambda x: x==4].index
indexes_CP_5 = ALL_DATA['First Digit CP4'].loc[lambda x: x==5].index
indexes_CP_6 = ALL_DATA['First Digit CP4'].loc[lambda x: x==6].index
indexes_CP_7 = ALL_DATA['First Digit CP4'].loc[lambda x: x==7].index
indexes_CP_8 = ALL_DATA['First Digit CP4'].loc[lambda x: x==8].index
indexes_CP_9 = ALL_DATA['First Digit CP4'].loc[lambda x: x==9].index
indexes_DB = ALL_DATA['First Digit CP4'].loc[lambda x:   x!=-2].index #This may look strange, but it is to keep the code more simple and concise on the predicting loop


Load bi-encoder and cross-encoder

In [ ]:
model_cross = CrossEncoder('/home/cross_encoder tuned')
model = SentenceTransformer('/home/bi_encoder tuned')

Create dictionary for making predicting loop more clear

In [ ]:
DB_Dicts = {
  "1": DB_CP1_,
  "2": DB_CP2_,
  "3": DB_CP3_,
  "4": DB_CP4_,
  "5": DB_CP5_,
  "6": DB_CP6_,
  "7": DB_CP7_,
  "8": DB_CP8_,
  "9": DB_CP9_,
  "-1":DATABASE_Embeddings,
}


DB_Indexes ={
  "1": indexes_CP_1,
  "2": indexes_CP_2,
  "3": indexes_CP_3,
  "4": indexes_CP_4,
  "5": indexes_CP_5,
  "6": indexes_CP_6,
  "7": indexes_CP_7,
  "8": indexes_CP_8,
  "9": indexes_CP_9,
  "-1":indexes_DB,
}

Considerando TOP 10 Matches

In [ ]:
discovered_addresses = []
discovered_arteries  = []
discovered_doors = []
match_confidence = []
label = []
scores = []

number_of_records_retrieved = 10

for i in tqdm(range(len(test_data))):

        firstCP4_digit = test_data.at[i, "First Digit CP4"]

        
        encode_real = model.encode(test_data.at[i, 'Morada_Full']).astype(float)
        cos_sim = util.cos_sim(encode_real, DB_Dicts[str(firstCP4_digit)])
        values, indices = torch.topk(cos_sim,number_of_records_retrieved) # Return top-10 most probable matches

        for j in range(number_of_records_retrieved): #Iterate over the top-10 returned matches
                #Calculate cross_encoder score between unnormalized_address and returned_address_i
                scores.append(model_cross.predict([ALL_DATA.at[DB_Indexes[str(firstCP4_digit)][indices[0][j].item()], 'Morada_Full_Normalizada'], test_data.at[i, 'Morada_Full']]))


        #Append to auxiliar lists the records that will be written in the results file. We write the values corresponding to the highest scored pair in the top-10 retrieved.
        discovered_addresses.append(ALL_DATA.at[DB_Indexes[str(firstCP4_digit)][indices[0][np.argmax(scores)].item()], 'Morada_Full_Normalizada'])
        discovered_arteries.append(ALL_DATA.at[DB_Indexes[str(firstCP4_digit)][indices[0][np.argmax(scores)].item()], 'ID_ARTERIA'])
        discovered_doors.append(ALL_DATA.at[DB_Indexes[str(firstCP4_digit)][indices[0][np.argmax(scores)].item()], 'ID_PORTA'])
        match_confidence.append(np.max(scores))
        scores = []



test_data['Discovered Address'] = discovered_addresses
test_data['Discovered Artery'] = discovered_arteries
test_data['Discovered Door'] = discovered_doors
test_data['Match Confidence'] = match_confidence

test_data.to_csv('results.csv', encoding='utf-8')